In [1]:
from google.cloud import storage
storage_client = storage.Client()

In [2]:
bucket_name = "ml-files-img"  # Replace with your bucket name
bucket = storage_client.get_bucket(bucket_name)

In [3]:
blobs = bucket.list_blobs()
#for blob in blobs:
#    print(blob.name)

In [4]:
#!rm -r /home/jupyter/images
#!rm -r /home/jupyter/proj

In [5]:
local_directory = "proj"  # Replace with your desired local directory

In [6]:
#%%time
#
#import os
#
#os.makedirs(local_directory, exist_ok=True)
#
## List all the objects in the bucket
#blobs = bucket.list_blobs()
#
## Iterate over the objects and download the images
#for blob in blobs:
#    if blob.name.endswith(".bmp") or blob.name.endswith(".jpg") or blob.name.endswith(".png"):
#        # Construct the local file path
#        local_file_path = os.path.join(local_directory, os.path.basename(blob.name))
#        # Download the image file
#        blob.download_to_filename(local_file_path)

In [7]:
#%%time
#
#import os
#
#os.makedirs(local_directory, exist_ok=True)
#
## List all the objects in the bucket
#blobs = bucket.list_blobs()
#
## Iterate over the objects and download the images
#for blob in blobs:
#    if blob.name.endswith(".csv"):
#        # Construct the local file path
#        local_file_path = os.path.join(local_directory, os.path.basename(blob.name))
#        # Download the image file
#        blob.download_to_filename(local_file_path)

In [8]:
#!pip install tensorflow
#!pip install sklearn

In [9]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
#!pip install keras_vggface
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

2023-05-20 18:16:36.100473: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 18:16:38.878129: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-20 18:16:38.878230: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/

In [11]:
# Load the BMI dataset
data = pd.read_csv("data.csv")
data.head()

,Unnamed: 0,bmi,gender,is_training,name
0,0,34.207396,Male,1,img_0.bmp
1,1,26.453720,Male,1,img_1.bmp
2,2,34.967561,Female,1,img_2.bmp
3,3,22.044766,Female,1,img_3.bmp
4,4,37.758789,Female,1,img_4.bmp


In [12]:
print(len(data))

4206


In [13]:
print(len(data[data['is_training']==1]))
print(len(data[data['is_training']==0]))

3368
838


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4206 entries, 0 to 4205
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4206 non-null   int64  
 1   bmi          4206 non-null   float64
 2   gender       4206 non-null   object 
 3   is_training  4206 non-null   int64  
 4   name         4206 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 164.4+ KB


In [15]:
#data.value_counts()

In [16]:
missing_list=['img_4.bmp',
 'img_5.bmp',
 'img_40.bmp',
 'img_86.bmp',
 'img_93.bmp',
 'img_112.bmp',
 'img_113.bmp',
 'img_238.bmp',
 'img_239.bmp',
 'img_300.bmp',
 'img_301.bmp',
 'img_309.bmp',
 'img_436.bmp',
 'img_437.bmp',
 'img_530.bmp',
 'img_531.bmp',
 'img_579.bmp',
 'img_617.bmp',
 'img_670.bmp',
 'img_671.bmp',
 'img_734.bmp',
 'img_735.bmp',
 'img_742.bmp',
 'img_820.bmp',
 'img_824.bmp',
 'img_860.bmp',
 'img_861.bmp',
 'img_1036.bmp',
 'img_1037.bmp',
 'img_1070.bmp',
 'img_1099.bmp',
 'img_1121.bmp',
 'img_1127.bmp',
 'img_1164.bmp',
 'img_1172.bmp',
 'img_1173.bmp',
 'img_1300.bmp',
 'img_1301.bmp',
 'img_1324.bmp',
 'img_1325.bmp',
 'img_1326.bmp',
 'img_1327.bmp',
 'img_1472.bmp',
 'img_1473.bmp',
 'img_1524.bmp',
 'img_1525.bmp',
 'img_1612.bmp',
 'img_1613.bmp',
 'img_1618.bmp',
 'img_1619.bmp',
 'img_1704.bmp',
 'img_1718.bmp',
 'img_1719.bmp',
 'img_1720.bmp',
 'img_1721.bmp',
 'img_1822.bmp',
 'img_1823.bmp',
 'img_1830.bmp',
 'img_1846.bmp',
 'img_1847.bmp',
 'img_1848.bmp',
 'img_1849.bmp',
 'img_1862.bmp',
 'img_1863.bmp',
 'img_1886.bmp',
 'img_1887.bmp',
 'img_1954.bmp',
 'img_1955.bmp',
 'img_1972.bmp',
 'img_1973.bmp',
 'img_2072.bmp',
 'img_2073.bmp',
 'img_2080.bmp',
 'img_2081.bmp',
 'img_2100.bmp',
 'img_2101.bmp',
 'img_2130.bmp',
 'img_2214.bmp',
 'img_2215.bmp',
 'img_2238.bmp',
 'img_2239.bmp',
 'img_2242.bmp',
 'img_2243.bmp',
 'img_2244.bmp',
 'img_2245.bmp',
 'img_2250.bmp',
 'img_2251.bmp',
 'img_2284.bmp',
 'img_2292.bmp',
 'img_2293.bmp',
 'img_2294.bmp',
 'img_2295.bmp',
 'img_2322.bmp',
 'img_2323.bmp',
 'img_2327.bmp',
 'img_2468.bmp',
 'img_2469.bmp',
 'img_2516.bmp',
 'img_2531.bmp',
 'img_2532.bmp',
 'img_2566.bmp',
 'img_2567.bmp',
 'img_2586.bmp',
 'img_2587.bmp',
 'img_2652.bmp',
 'img_2653.bmp',
 'img_2666.bmp',
 'img_2667.bmp',
 'img_2712.bmp',
 'img_2714.bmp',
 'img_2715.bmp',
 'img_2722.bmp',
 'img_2730.bmp',
 'img_2731.bmp',
 'img_2738.bmp',
 'img_2739.bmp',
 'img_2750.bmp',
 'img_2751.bmp',
 'img_2785.bmp',
 'img_2842.bmp',
 'img_2843.bmp',
 'img_2912.bmp',
 'img_2913.bmp',
 'img_2954.bmp',
 'img_2955.bmp',
 'img_2982.bmp',
 'img_2983.bmp',
 'img_2986.bmp',
 'img_2987.bmp',
 'img_3030.bmp',
 'img_3031.bmp',
 'img_3039.bmp',
 'img_3070.bmp',
 'img_3071.bmp',
 'img_3084.bmp',
 'img_3085.bmp',
 'img_3103.bmp',
 'img_3140.bmp',
 'img_3141.bmp',
 'img_3146.bmp',
 'img_3152.bmp',
 'img_3153.bmp',
 'img_3188.bmp',
 'img_3189.bmp',
 'img_3268.bmp',
 'img_3269.bmp',
 'img_3284.bmp',
 'img_3285.bmp',
 'img_3298.bmp',
 'img_3299.bmp',
 'img_3320.bmp',
 'img_3321.bmp',
 'img_3336.bmp',
 'img_3337.bmp',
 'img_3340.bmp',
 'img_3341.bmp',
 'img_3366.bmp',
 'img_3367.bmp',
 'img_3368.bmp',
 'img_3376.bmp',
 'img_3377.bmp',
 'img_3387.bmp',
 'img_3394.bmp',
 'img_3404.bmp',
 'img_3405.bmp',
 'img_3408.bmp',
 'img_3409.bmp',
 'img_3424.bmp',
 'img_3425.bmp',
 'img_3430.bmp',
 'img_3431.bmp',
 'img_3498.bmp',
 'img_3499.bmp',
 'img_3503.bmp',
 'img_3504.bmp',
 'img_3505.bmp',
 'img_3530.bmp',
 'img_3531.bmp',
 'img_3570.bmp',
 'img_3571.bmp',
 'img_3590.bmp',
 'img_3591.bmp',
 'img_3630.bmp',
 'img_3631.bmp',
 'img_3656.bmp',
 'img_3657.bmp',
 'img_3660.bmp',
 'img_3680.bmp',
 'img_3681.bmp',
 'img_3694.bmp',
 'img_3695.bmp',
 'img_3726.bmp',
 'img_3727.bmp',
 'img_3731.bmp',
 'img_3764.bmp',
 'img_3765.bmp',
 'img_3790.bmp',
 'img_3792.bmp',
 'img_3793.bmp',
 'img_3794.bmp',
 'img_3795.bmp',
 'img_3802.bmp',
 'img_3803.bmp',
 'img_3812.bmp',
 'img_3813.bmp',
 'img_3816.bmp',
 'img_3826.bmp',
 'img_3840.bmp',
 'img_3841.bmp',
 'img_3851.bmp',
 'img_3854.bmp',
 'img_3855.bmp',
 'img_3878.bmp',
 'img_3879.bmp',
 'img_3880.bmp',
 'img_3881.bmp',
 'img_3900.bmp',
 'img_3906.bmp',
 'img_3907.bmp',
 'img_3921.bmp',
 'img_3922.bmp',
 'img_3923.bmp',
 'img_3931.bmp',
 'img_3958.bmp',
 'img_3966.bmp',
 'img_3967.bmp',
 'img_3970.bmp',
 'img_3971.bmp',
 'img_3973.bmp',
 'img_3990.bmp',
 'img_3991.bmp',
 'img_3994.bmp',
 'img_3995.bmp',
 'img_4018.bmp',
 'img_4019.bmp',
 'img_4104.bmp',
 'img_4105.bmp',
 'img_4146.bmp',
 'img_4147.bmp',
 'img_4168.bmp',
 'img_4174.bmp',
 'img_4175.bmp',
 'img_4182.bmp',
 'img_4183.bmp']

In [17]:
data_filtered = data[~data['name'].isin(missing_list)]

In [18]:
data = data_filtered
len(data)

3962

In [19]:
print(len(data[data['is_training']==1]))
print(len(data[data['is_training']==0]))

3210
752


In [20]:
train_data = data[data["is_training"] == 1]
test_data = data[data["is_training"] == 0]

In [21]:
train_paths = train_data["name"].tolist()
train_labels = train_data["bmi"].tolist()

val_paths = test_data["name"].tolist()
val_labels = test_data["bmi"].tolist()

In [22]:
print(len(train_paths))
print(len(val_paths))

3210
752


In [23]:
print(len(train_labels))
print(len(val_labels))

3210
752


#### Adding Custom Layers: The code adds custom layers on top of the VGG16 model. It adds a global average pooling layer to reduce the spatial dimensions, followed by a fully connected (dense) layer with ReLU activation function, and an output layer with a single neuron for BMI prediction.

In [24]:
#!pip install keras_applications
#!pip install keras-vggface

In [25]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2023-05-20 18:16:42.217606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-20 18:16:42.229167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-20 18:16:42.230828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [26]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
import tensorflow.keras.mixed_precision as mixed_precision

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable mixed precision training
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


2023-05-20 18:16:42.243911: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [27]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 128 neurons and L2 regularization
x = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)

# Add a fully connected layer with 128 neurons and L2 regularization
x = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)

# Add a fully connected layer with 128 neurons and L2 regularization
x = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)

# Add a fully connected layer with 128 neurons and L2 regularization
x = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)

# Add an output layer with 1 neuron for BMI prediction
predictions = Dense(1)(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze the last 5 layers of the pre-trained VGG16 model and apply L2 regularization
for layer in model.layers[-5:]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        # Apply L2 regularization to convolutional layers
        layer.kernel_regularizer = regularizers.l2(0.01)
    if isinstance(layer, tf.keras.layers.Dense):
        # Apply L2 regularization to dense layers
        layer.kernel_regularizer = regularizers.l2(0.01)
    layer.trainable = True
for layer in model.layers[:15]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        # Apply L2 regularization to convolutional layers
        layer.kernel_regularizer = regularizers.l2(0.01)
    if isinstance(layer, tf.keras.layers.Dense):
        # Apply L2 regularization to dense layers
        layer.kernel_regularizer = regularizers.l2(0.01)
    layer.trainable = False

# Compile the model with RMSprop optimizer and mean absolute error loss
model.compile(optimizer="RMSprop", loss="mean_absolute_error")

2023-05-20 18:16:42.271256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 18:16:42.271700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-20 18:16:42.273619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-20 18:16:42.275264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at 

In [28]:
%%time

from tensorflow.keras.callbacks import EarlyStopping

# Define the image data generator with data augmentation
datagen = image.ImageDataGenerator(rescale=1.0 / 255.0,  # Scale pixel values to [0, 1]
                                   rotation_range=20,  # Randomly rotate images by 20 degrees
                                   width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
                                   height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
                                   shear_range=0.2,  # Apply random shear transformations
                                   zoom_range=0.2,  # Apply random zoom transformations
                                   horizontal_flip=True)  # Randomly flip images horizontally

# Create training and validation data generators
train_generator = datagen.flow_from_dataframe(
    pd.DataFrame({"path": train_paths, "BMI": train_labels}),
    x_col="path",
    y_col="BMI",
    target_size=(224, 224),
    batch_size=16,
    class_mode="raw",
)
val_generator = datagen.flow_from_dataframe(
    pd.DataFrame({"path": val_paths, "BMI": val_labels}),
    x_col="path",
    y_col="BMI",
    target_size=(224, 224),
    batch_size=16,
    class_mode="raw",
)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

# Train the model with early stopping
history = model.fit(train_generator, validation_data=val_generator, epochs=30, callbacks=[early_stopping])

Found 3210 validated image filenames.
Found 752 validated image filenames.
Epoch 1/30


2023-05-20 18:16:46.270830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-05-20 18:16:49.456607: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x559de3a50d20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-20 18:16:49.456669: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-05-20 18:16:49.465166: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-20 18:16:49.465470: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-20 18:16:49.673237: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


201/201 [==============================] - 48s 208ms/step - loss: 52.0342 - val_loss: 10.7847
Epoch 2/30
201/201 [==============================] - 40s 201ms/step - loss: 12.0872 - val_loss: 20.5770
Epoch 3/30
201/201 [==============================] - 40s 200ms/step - loss: 10.5155 - val_loss: 12.7034
Epoch 4/30
201/201 [==============================] - 41s 202ms/step - loss: 9.9762 - val_loss: 11.6333
Epoch 5/30
201/201 [==============================] - 41s 202ms/step - loss: 9.3538 - val_loss: 13.6250
Epoch 6/30
201/201 [==============================] - 40s 200ms/step - loss: 9.0410 - val_loss: 7.7955
Epoch 7/30
201/201 [==============================] - 41s 203ms/step - loss: 8.5302 - val_loss: 11.5785
Epoch 8/30
201/201 [==============================] - 41s 203ms/step - loss: 8.3333 - val_loss: 11.6225
Epoch 9/30
201/201 [==============================] - 40s 199ms/step - loss: 7.9170 - val_loss: 8.7065
Epoch 10/30
201/201 [==============================] - 40s 201ms/step - lo

In [ ]:
# Save the model
model.save("bmi_mode_2005_1.h5")

In [30]:
%%time

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# Calculate predictions for the validation set
val_predictions = model.predict(val_generator)
val_predictions = val_predictions.flatten()

# Create a dataframe for actual and predicted BMI values
val_results = pd.DataFrame({'Actual BMI': val_labels, 'Predicted BMI': val_predictions})

# Add the image file names column
val_results['Image File'] = val_paths

47/47 [==============================] - 8s 164ms/step
CPU times: user 9.8 s, sys: 208 ms, total: 10 s
Wall time: 8.17 s


In [31]:
val_results.head(20)

,Actual BMI,Predicted BMI,Image File
0,29.698495,29.906250,img_3369.bmp
1,30.845918,36.062500,img_3370.bmp
2,24.389796,28.765625,img_3371.bmp
3,36.258679,32.406250,img_3372.bmp
4,27.891291,28.812500,img_3373.bmp
5,36.743467,28.593750,img_3374.bmp
6,23.370844,29.406250,img_3375.bmp
7,60.096828,28.046875,img_3378.bmp
8,34.256669,28.359375,img_3379.bmp
9,28.884838,35.968750,img_3380.bmp


In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate predictions for the validation set
val_predictions = model.predict(val_generator)
val_predictions = val_predictions.flatten()

# Calculate RMSE
val_rmse = np.sqrt(mean_squared_error(val_labels, val_predictions))

# Calculate MAE
val_mae = mean_absolute_error(val_labels, val_predictions)

print("Validation RMSE:", val_rmse)
print("Validation MAE:", val_mae)

47/47 [==============================] - 8s 161ms/step
Validation RMSE: 10.619774783004708
Validation MAE: 7.79690765935625


In [33]:
from sklearn.metrics import roc_auc_score

# Convert actual BMI values to binary labels (0 or 1)
actual_labels = pd.cut(val_results['Actual BMI'], bins=[0, 24.9, np.inf], labels=[0, 1])

# Calculate AUC
auc = roc_auc_score(actual_labels, val_results['Predicted BMI'])

# Print AUC
print("AUC:", auc)

AUC: 0.46475818694426374


In [34]:
from sklearn.metrics import accuracy_score, r2_score

# Calculate R2 score
val_r2 = r2_score(val_results['Actual BMI'], val_results['Predicted BMI'])
print("R2 Score:",val_r2)

R2 Score: -0.3460724925824106


In [35]:
import pandas as pd

# Assuming you have a DataFrame called 'df'
# ...

# Save the DataFrame as a CSV file
val_results.to_csv('predictions.csv', index=False)
